# 生物先端コース 学生実習 2025後期 2日目

## NGSショートリードマッピング・Variant calling

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/mutation.png?raw=true" alt="title" height="150px">

本実習では、Genomics, Transcriptomics, Proteomicsに関する基本的なゲノム解析技術を学習していきます。

2日目は、ゲノム上にある遺伝的変異(SNP等)を検出する方法について学びます。

遺伝的変異データは、GWASや系統解析など様々な解析に必要なデータとなります。


# 遺伝的変異について

遺伝的変異とは、生物集団内や個体間で見られるDNA配列の差異のことを指します。

この様な差異によって、生き物の持つ表現型に違いが生み出されています。

例えば同じイネの"こしひかり"と"あきたこまち"も、ほとんどDNA配列は同じですが、

一部の配列が異なることで味や見た目が異なっています。

つまり、こしひかりとあきたこまちのDNA配列を比較し、違いを見つける事が出来れば味の違いを生み出す原因遺伝子等を見つける事が出来る、ということになります。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/mutation2.png?raw=true" alt="title" height="200px">

この様なアプローチを**順遺伝学**と呼びます。

逆にゲノム編集等で遺伝子を壊して形質がどう変化するか見るアプローチを**逆遺伝学**と呼びます。

今回はこの順遺伝学を進めるために必須となる遺伝的変異の検出方法の実習を行います。

# 次世代シーケンサーによる変異検出

1日目はゲノムアセンブルという技術を使って、ゲノム全体のDNA配列を解読しました。

では遺伝的変異を検出するためには毎回アセンブルをして全ゲノムを復元して比較するのかというと、それは現実的では無いです。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/bad_strategy.png?raw=true" alt="title" height="300px">

数個体なら何とかなりますが、交配集団100個体全ゲノム復元…等の話になるとコストがかかり過ぎます。

また、先ほど出した例のように、こしひかりとあきたこまちを比較する場合、殆どのDNA配列は同じで一部だけ異なっていると想定されるので、毎回似たような配列を全部読むのも非効率です。

## NGS(次世代シーケンサー)を利用

そこで、前回扱ったロングリードとは異なる、ショートリードを読む事が出来る次世代シーケンサーを用います。

ショートリードは現在最もよく使用されるシーケンスになり、100~300塩基ほどの長さのリードを安価で大量に読む事が出来ます。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/Basic_4.png?raw=true" alt="title" height="270px">

この大量に読んだショートリードをある特定のリファレンスゲノムに対し貼り付けていき(**マッピング**)、遺伝的変異を探していく、

というのが遺伝的変異を検出する一般的な手法になります。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/Basic_6.png?raw=true" alt="title" height="350px">

# 遺伝的変異検出の実習

それでは、昨日作成したいもち病菌(**GZ菌株**)のリファレンスゲノムと比べて、新しく採集したいもち病菌(**d10a菌株**)が持つ遺伝的変異を検出する、という解析を実際に実施していきましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/magnaporthe.png?raw=true" alt="title" height="300px">

## シーケンス結果の取得

まずはコマンド操作の復習がてら、データセットのダウンロードをしていきます。

昨日実行したGZ菌株のアセンブルの結果であるゲノム配列は

* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/assembly.fasta

いもち病菌(d10a菌株)のDNA配列を次世代シーケンサーで読んだ配列データは

* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/d10a_1.fastq.gz
* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/d10a_2.fastq.gz

参考用に昨日扱ったDNA配列をロングリードで読んだ配列データは

* https://github.com/slt666666/NLR_biology_workshop_2024/raw/master/data/bctg.part_001.fastq.gz

にあります。左下からターミナルを開いて`wget`コマンドで上の4つのデータをダウンロードしてみてください。

```
wget <ダウンロードしたいファイルのURL>
```

また、配列データの特徴を確認する`seqkit`もインストールしておきます。

```
apt install seqkit
```

ダウンロードした配列を`less`コマンドや`seqkit stat`コマンドを使用して見てみると、

今回扱うショートリードと昨日扱ったロングリードの違いがよくわかると思います。

```
less -S <配列データファイル名>
```

```
seqkit stats <配列データファイル名>
```
また、d10a系統のDNAを読んだショートリードのデータには2つのファイル`d10a_1.fastq.gz`と`d10a_2.fastq.gz`があります。

これは、ショートリードを読むときに`pair-end`というやり方で読んでいるためです。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/paired_end.png?raw=true" alt="title" height="150px">

## 配列データのマッピング

では続いて、ショートリードとして読んだDNA配列をリファレンスゲノムに貼り付けていきます。

この操作を**マッピング**とか**アライメント**と呼びます。

<img src="https://github.com/slt666666/FAO_lecture/blob/main/FAO_2024/Basic_6.png?raw=true" alt="title" height="350px">

マッピングをするためのプログラムには様々なものが存在し、

正確性の向上や高速化や効率化を目指して日々新たなツールが開発されています。

* [BWA](https://github.com/lh3/bwa)
* [BWA-mem2](https://github.com/bwa-mem2/bwa-mem2)
* [Bowtie2](https://bowtie-bio.sourceforge.net/bowtie2/index.shtml)

などが代表的なプログラムになります。

今回は最もベーシックな[BWA](https://github.com/lh3/bwa)というプログラムを使用します。

下のコマンドで`bwa`プログラムをインストールしましょう。
```
apt install bwa
```


### リファレンスゲノムのindex作成

まずマッピングする前にリファレンスゲノムのindexファイルというものを作成します。

indexファイルとは、**索引ファイル**のことを指し、これを作成しておくことで、

`bwa`プログラムが染色体名や長さ、ファイル内の位置等を瞬時に参照できるようにします。

(あんまり理解しなくて大丈夫です。お作法みたいなものです。)

indexファイルは下のコマンドで作成できます。

```
bwa index assembly.fasta
```


### マッピングの開始

indexファイルを作成したら、実際にショートリードのマッピングをしていきます。

`bwa mem`コマンドでマッピングができます。

```
bwa mem <リファレンスゲノムのファイル名> <ショートリード(R1)ファイル名> <ショートリード(R2)ファイル名> > <出力ファイル名.sam>
```

今回はリファレンスが`assembly.fasta`、それぞれのショートリードが`d10a_1.fastq.gz, d10a_2.fastq.gz`になるので、
```
bwa mem assembly.fasta d10a_1.fastq.gz d10a_2.fastq.gz > d10a.sam
```
というコマンドになります。

コマンド中に出てくる`>`ですが、`コマンド > 出力ファイル名`と記述すると、左側のコマンドを実施した結果を右側の出力ファイル名に書き込むことが出来ます。

(~5分くらいかかります)

プログラムが走り終えると、`d10a.sam`というファイルが出来ているかと思います。

#### **SAMファイル/BAMファイル**

SAMファイルは**Sequence Alignment and Map**ファイルのことを指し、

今回使用したショートリードのマッピング結果の情報がテキストとして保存されています。

どのリードが、どの場所にマッピングされ、そのクオリティがどの程度なのか…等の情報を見る事が出来ます。

`less`コマンドで見てみましょう。

```
less -S <SAMファイル名>
```
しかし、SAMファイルはテキストで全ての情報を記述しているので、ファイルサイズが大きくなってしまいます。

また、リード毎に情報が載っているので、位置は色んな場所が入り乱れています。

そのため、1番染色体のこのあたりに張り付いたリード…の様な情報を得ることが難しいです。

そこでこのSAMファイルを操作するためのプログラム`samtools`を使用して整理します。

```
apt install samtools
```

samtoolsがインストール出来たら、`samtools sort`というコマンドで、

SAMファイルの中身をゲノム上の位置に従ってソートする事が出来ます。

また、オプションで`-O bam`とつけることで、SAM形式から**BAM**(**Binary Alignment and Map**)形式に変換できます。

```
samtools sort -O bam <SAMファイル名> > <出力ファイル名.bam>
samtools index <BAMファイル名>
```
下の二つのコマンドを動かしてみてください。
```
samtools sort -O bam d10a.sam > d10a.bam
samtools index d10a.bam
```

BAM形式はテキストではなくバイナリー形式(コンピュータのみ理解できる)で情報が保存されているので、データサイズを小さくすることができます。

集団の解析では大量のリードをマッピングした情報を個体ごとには生成することになり、サイズが非常に大きくなるので、基本的にBAMファイルとしてマッピング結果は保存することが多いです。


### マッピング結果の可視化

これでBAM(SAM)ファイルとして、大量のショートリードをマッピングした結果が得られました。

マッピング結果は、文字で見てもあんまりよくわからないですが、可視化することで分かりやすくなります。

マッピング結果の可視化には`IGV`というプログラムがよく使用されます。

皆さんのPCにデスクトップ版のIGVをダウンロードしてもらっても良いのですが、ちょっと重いソフトになるので今回はGoogle Colab上のプログラムとして動かします。

下の3つのコマンドを動かすと、Google Colab上でIGVが使用可能になります。

```
pip install -q igv-notebook==0.3.1
wget -q -O modules.py https://raw.githubusercontent.com/slt666666/FAO_lecture/main/FAO_2024/data/modules.py

samtools faidx assembly.fasta
```
プログラムのダウンロードが終わったら、下のプログラムを動かすことでIGVが動きます。

In [ ]:
## Visualize alignment results
## import libraries
import igv_notebook
from modules import RefTrack, AnnotationTrack, BamTrack
igv_notebook.init()
# show reference genome
ref = RefTrack({ "fastaPath":"assembly.fasta", "indexPath":"assembly.fasta.fai", "id":"Strain_X" })
# show aligned reads
B = BamTrack({ "name":"d10a", "path":"d10a.bam", "indexPath":"d10a.bam.bai", "viewAsPairs":True })
# visualize
b = igv_notebook.Browser(ref)
b.load_track(B)


可視化してみると、どの場所にどんな変異が入っているのかよく分かります。

リファレンスゲノムに対して、張り付いているリードは新しいいもち病菌由来のものなので、

一部のDNA配列が異なっていれば、そこに変異が見つかる形です。

### Variant call

さすがに目視で全ての変異をチェックすることは難しいので、マッピング結果のBAMファイルから変異の情報を取り出すプログラムを使用します。

その様な、BAMファイルから変異を検出するプログラムを**Variant caller**と呼びます。

有名なVariant callerとしては[bcftools](https://samtools.github.io/bcftools/bcftools.html)や[GATK](https://gatk.broadinstitute.org/hc/en-us)があります。

今回は`bcftools`を使用するので、今までと同じようにインストールしましょう。

```
apt install bcftools
```

`bcftools`にはいろんな関数や機能がありますが、BAMファイルから変異を検出するには

`bcftools mpileup`と`bcftools call`というコマンドを使用します。

まず、`bcftools mpileup`というコマンドを使用して、VCFファイルというものを作成します。

VCFとはvariant call formatを意味します。

```
bcftools mpileup -Ov -o <VCFファイル名> -f <リファレンスゲノムのファイル名> <BAMファイル名>
```
今回は`d10a.vcf`という名前のファイルに出力してみましょう。

このVCFファイルには、マッピングされた結果から得られる、d10a系統の遺伝子型の情報が入っています。

ただし、変異の無い場所の情報も含まれてしまっているので、`bcftools call`というコマンドを使用して、VCFファイルの中から、変異の部分だけを取り出します。

```
bcftools call --ploidy 1 -vmO v -o <出力VCFファイル名> <入力VCFファイル名>
```
今回は`d10a_variants.vcf`という名前のファイルに出力してみましょう。

また、いもち病菌はHaploid(1倍体)なので、そのオプションもつけています。

このコマンドを動かすと、`d10a_variants.vcf`というファイルが作成され、今回のマッピングによって検出された遺伝的変異の一覧が手に入りました。

VCFファイルにはどの染色体のどの位置の配列にどんな風に変異が入っているのかが記されています。

これまでの流れをまとめると、

1. 基準となるリファレンスゲノムと、どんな遺伝的変異を持つか知りたい生物のシーケンス結果(ショートリード)を用意
2. ショートリードをリファレンスゲノムにマッピングする(`bwa` & `samtools`)
3. マッピング結果から遺伝的変異を検出する(`bcftools`)

となります。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/flowchart.png?raw=true" alt="title" height="250px">

### 複数サンプルの遺伝的変異データ

ここまで、SNP等の遺伝的変異を検出する一般的な方法を扱いましたが、

通常の研究では集団全体に対してこのような変異を検出し、解析する場合が多いです。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/population_example.png?raw=true" alt="title" height="250px">

そこで、複数サンプルの遺伝的変異の扱いを見てみます。

リファレンス配列を読んだ菌株GZ、先ほどまで扱っていた菌株d10aに加え、

新しいいもち病菌の菌株d21aのショートリードを追加で読んだとします。

いもち病菌(菌株d21a)のDNA配列を次世代シーケンサーで読んだ配列データは

* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/d21a_1.fastq.gz
* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/d21a_2.fastq.gz

にあります

`wget`コマンドでデータをダウンロードし、このショートリード(`d21a_1.fastq.gz`, `d21a_2.fastq.gz`)を使用して、

復習がてら、さきほどと同じ過程で菌株d21aについて、マッピング結果の`BAMファイル`を作成してみてください。

1. マッピング
```
bwa mem <リファレンスゲノムのファイル名> <ショートリード(1)ファイル名> <ショートリード(2)ファイル名> > <出力ファイル名>
```
2. マッピング結果のソート、BAM変換
```
samtools sort -O bam <SAMファイル名> > <出力ファイル名.bam>
samtools index <BAMファイル名>
```

`d21a.bam`と`d21a.bam.bai`という2つのファイルが新たに出来ていれば成功です。


続いて、菌株d10aと菌株d21aにある変異をまとめて検出します。

`bcftools mpileup`では複数のBAMファイルを扱うことが可能です。

また、リードの張り付いた数をサンプル毎に計測する`--annotate FORMAT/DP`というオプションもつけています。

```
bcftools mpileup --annotate FORMAT/DP -Ov -o <VCFファイル名> -f <リファレンスゲノムのファイル名> <BAMファイル名1> <BAMファイル名2> ...
```

最後に指定するBAMファイルを`d10a.bam`と`d21a.bam`の2つ指定して動かしてみてください。

`-o`で指定する出力するVCFファイル名は`two.vcf`としておきます。

```
bcftools mpileup --annotate FORMAT/DP -Ov -o three.vcf -f assembly.fasta <BAMファイル1> <BAMファイル2>
```

続いて、このVCFファイルから遺伝的変異のみを`bcftools call`コマンドで抽出します。

```
bcftools call --ploidy 1 -vmO v -o <出力VCFファイル名> <入力VCFファイル名>
```
出力VCFファイル名は`two_variant.vcf`とします。

出力された`two_variant.vcf`というファイルを`less -S`コマンドなどで見てみると、

菌株`d10a`または`d21a`いずれかに変異がある場所が全てとりあげられています。

1サンプルの時は変異が入った場所のみ表示されていましたが、今回は

* 菌株`d10a`は変異しているが`d21a`はしていない

* `d10a`では遺伝子型が分かるが、`d21a`ではリードが読まれていないので遺伝子型が不明

などのケースも含まれています。

そのため、各サンプルの遺伝子型を、`0`はリファレンス通り、`1`は変異あり、`.`はリードが張り付かず情報なし、という形で表しています。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/vcf.png?raw=true" alt="title" height="250px">

ちなみに遺伝子型の部分、今回は1倍体なので0 or 1ですが、2倍体の場合0/0 or 0/1 or 1/1となります。



### 検出した変異のフィルタリング

こうして見ると、片方のサンプルの遺伝子型情報が無かったり、あったとしても張り付いたリード数が少なく、信頼性が低い場所などがあることがわかります。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/vcf_qual.png?raw=true" alt="title" height="150px">

そこで、通常は作成したVCFファイルをそのまま使用するのではなく、**VCFのフィルタリング**という作業を行います。

得られた変異のデータの中で、信頼性の高い場所の情報だけ取り出す作業です。

このVCFのフィルタリングには、`vcftools`というプログラムが便利です。
(`bcftools`ではなく、`vcftools`です。)

下記コマンドで`vcftools`をインストールします。

```
apt install vcftools
```

#### **リードのデプスが基準を下回る変異を削除**

何本のリードによって遺伝子型が検出されたのか、この張り付いたリードの数をデプスと呼びます。

デプスが大きければ信頼性が高く、小さければシーケンスエラーの可能性が高くなります。

そこで、このデプスが5以上の部分だけ取り出す、みたいな事が出来ます。

```
vcftools --vcf 入力VCFファイル名 --minDP 5 --recode --out minDP5
```

実行すると`minDP5.recode.vcf`というファイルが出力されます


#### **Genotyping率が基準を下回る変異を削除**

今回は2サンプルあるので、全サンプル(100%)遺伝子型が決まっている場所か、1サンプルだけ(50%)遺伝子型が決まっている場所か、という風に場合分けする事が出来ます。

集団の解析をしていると、集団の30%の個体しか遺伝子型が決まっていない場所や集団の90%の個体の遺伝子型が決まっている場所など、

各変異ごとに、どれだけの個体について遺伝子型が決まっているか異なります。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/missing_SNP.png?raw=true" alt="title" height="100px">

そこで、集団中のある程度の個体の遺伝子型が決まっている場所だけ抽出する、という事が出来ます。

集団の80%の個体がちゃんと遺伝子型が決まっている場所を取り出す場合は、

```
vcftools --vcf 入力VCFファイル名 --max-missing 0.8 --recode --out minDP5_mm0.8
```

実行すると`minDP5_mm0.8.recode.vcf`というファイルが出力されます。

この様な形で得られた遺伝的変異の中でもより信頼性の高い情報のみを取り出していきます。

他にも、VCFファイルのクオリティスコアを基準に選抜することもできたりします。

ここからは実際に検出した変異の情報を使用して、系統解析やGWASを実施してみます。

これらの解析のために、20系統のDNAからショートリードを取得し、マッピングなどを実施して作成したVCFがこちらになります。

* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/population.vcf

こちらのファイルについても`wget`コマンドでダウンロードし、先ほどと同様デプスが5以上と集団の80%の個体が遺伝子型ありの条件でフィルタリングしてみてください。

```
vcftools --vcf 入力VCFファイル名 --minDP 5 --recode --out minDP5
vcftools --vcf 入力VCFファイル名 --max-missing 0.8 --recode --out minDP5_mm0.8
```


## 系統解析

集団の遺伝的変異情報を得る事が出来ると、その進化的な関係性を推測し、系統解析を行う事が出来ます。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/phylogeny.png?raw=true" alt="title" height="200px">

系統解析を行うプログラムにはRAxMLやMEGA、IQ-Treeなど様々なものがありますが、

今回はWebアプリとして実装されているIQ-Treeを使用します。

http://iqtree.cibiv.univie.ac.at/

系統解析を実施するためには、変異データであるVCFファイルを、使用するプログラムに応じて変異の配列データなどに変換する必要があります。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/vcf_convert.png?raw=true" alt="title" height="150px">


IQ-Treeは`Phylip`形式と呼ばれるファイルを必要とするので、まずは`vcf2phylip`というプログラムを使用してVCFファイルをPhylipファイルに変換します、

`vcf2phylip`のインストール
```
git clone https://github.com/edgardomortiz/vcf2phylip.git
```

VCFファイルの変換コマンド
```
python vcf2phylip/vcf2phylip.py -i minDP5_mm0.8.recode.vcf
```

このコマンドを動かすと`minDP5_mm0.8.recode.min4.phy`というPHYLIPファイルが作成されます。

あとはIQ-Treeのサイトに作成したPHYLIPファイルをアップロードして、系統解析を実行してみましょう。


<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/iqtree.png?raw=true" alt="title" height="350px">

実行し終えると、メニューのFull resultsの下の方に系統樹のデータがあります。

Consensus tree in newick formatという所です。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/newick.png?raw=true" alt="title" height="350px">

この部分をコピーして、[iTOL](https://itol.embl.de/)のUploadから系統樹を綺麗に表示させることもできます。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/itol.png?raw=true" alt="title" height="350px">

系統解析などを行う場合は、系統樹の推定ソフトで系統樹推定を行いiTOLで可視化、という流れが多いです。


## GWAS(Genome Wide Association Study)

集団に存在する形質の違いと関連する遺伝的変異を検出する手法の一つがGWASになります。

<img src="https://github.com/CropEvol/lecture/blob/master/images/exp_2025_2/gwas.png?raw=true" alt="title" height="300px">

詳しくは[ゲノム情報解析入門](https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2024/GWAS.ipynb#scrollTo=_oaBjJrGgxur)の講義資料を参照してください。

GWASを実施するには、集団の遺伝的変異の情報と、各個体の形質データが必要になります。

形質データ`Phenotype_for_GWAS.txt`を下記コマンドでダウンロードします。
```
wget https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/Phenotype_for_GWAS.txt
```

GWASを行うためのプログラムとしては、`PLINK`やRで使用可能な`rrBLUP`や`GWASPoly`など様々なプログラムがあります。

使用する統計モデルや考慮する遺伝的効果の種類などに応じてプログラムを選択する必要があります。

今回はPythonで使用できる[hail](https://hail.is/)というプログラムを使用します。

下記コマンドで`hail`をインストールできます。

```
pip install hail
```

In [ ]:
import hail as hl
from hail.plot import show

# 初期設定
hl.plot.output_notebook()

# リファレンスゲノムの設定
hl.genetics.ReferenceGenome.from_fasta_file(name="reference", fasta_file="assembly.fasta", index_file="assembly.fasta.fai")

# VCFファイルの読み込み
hl.import_vcf('minDP5_mm0.8.recode.vcf', reference_genome="reference").write('minDP5_mm0.8.recode.mt', overwrite=True)
mt = hl.read_matrix_table('minDP5_mm0.8.recode.mt')

# 形質データの読み込み
table = hl.import_table('Phenotype_for_GWAS.txt', impute=True).key_by('Line')
mt = mt.annotate_cols(pheno=table[mt.s])

# GWASの実行
gwas = hl.linear_regression_rows(y=mt.pheno.size, x=mt.GT.n_alt_alleles(), covariates=[1.0])

# Manhattan plotの描写
p = hl.plot.manhattan(gwas.p_value)
show(p)


この様な形で、集団の遺伝的変異を検出し、進化的な関係性を見たり、形質に関与する変異を探索します。

研究で順遺伝学的な手法を取り入れる場合は、このようなことを実施する可能性が高いです。

次回はRNA-seqを使用して遺伝子発現量の定量化を実施します。